In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 한식만

In [6]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = "data/수원시 한식 데이터백업.csv"
df = pd.read_csv(DATA_PATH)

# ===== (1) 날짜 컬럼 찾기/통일 =====
date_col = "DATE" if "DATE" in df.columns else ("TA_YMD" if "TA_YMD" in df.columns else None)
if date_col is None:
    raise ValueError("날짜 컬럼이 DATE 또는 TA_YMD로 존재해야 합니다.")

df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col).reset_index(drop=True)

TARGET = "AMT"
if TARGET not in df.columns:
    raise ValueError(f"타겟 컬럼 {TARGET} 이(가) 없습니다. 컬럼명 확인 필요.")

# ===== (2) DONG 문자열 처리 =====
cat_cols = []
if "DONG" in df.columns:
    df["DONG"] = df["DONG"].astype("category")
    cat_cols.append("DONG")

# ===== (3) 피처 엔지니어링 =====
def make_features(data):
    d = data.copy()

    # 날짜 파생 (datetime 제거 목적)
    d["year"] = d[date_col].dt.year
    d["month"] = d[date_col].dt.month
    d["day"] = d[date_col].dt.day
    d["dow"] = d[date_col].dt.weekday
    d["weekend"] = (d["dow"] >= 5).astype(int)

    # Lag / Rolling (AMT 기반)
    for lag in [1, 7, 14]:
        d[f"lag_{lag}"] = d[TARGET].shift(lag)

    for win in [7, 14]:
        d[f"roll_mean_{win}"] = d[TARGET].shift(1).rolling(win).mean()
        d[f"roll_std_{win}"] = d[TARGET].shift(1).rolling(win).std()

    return d

df = make_features(df)
df = df.dropna().reset_index(drop=True)

# ===== (4) 학습용 X/y 구성 =====
drop_cols = [TARGET, date_col]
FEATURES = [c for c in df.columns if c not in drop_cols]

X = df[FEATURES].copy()
y = df[TARGET].copy()

# LightGBM이 category dtype을 처리할 수 있도록 유지
# (단, 혹시 object 남아있으면 category로 바꿔줌)
for c in X.columns:
    if X[c].dtype == "object":
        X[c] = X[c].astype("category")
        if c not in cat_cols:
            cat_cols.append(c)

# ===== (5) TimeSeries CV =====
tscv = TimeSeriesSplit(n_splits=4)
mae_scores = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMRegressor(
        objective="regression_l1",
        n_estimators=2000,
        learning_rate=0.03,
        num_leaves=64,
        min_child_samples=30,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.3,
        n_jobs=-1,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="l1",
        categorical_feature=cat_cols if len(cat_cols) > 0 else "auto",
        callbacks=[lgb.early_stopping(200, verbose=False)]
    )

    pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, pred)
    mae_scores.append(mae)
    print(f"Fold {fold+1} MAE: {mae:,.0f}")

print("\n====================")
print(f"CV 평균 MAE: {np.mean(mae_scores):,.0f}")
print("====================")

# ===== (6) 최종 모델 =====
final_model = lgb.LGBMRegressor(
    objective="regression_l1",
    n_estimators=int(np.mean([model.best_iteration_ for _ in range(1)]) or 1500),
    learning_rate=0.03,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.3,
    reg_lambda=0.3,
    n_jobs=-1,
    random_state=42
)
final_model.fit(X, y, categorical_feature=cat_cols if len(cat_cols) > 0 else "auto")

print("최종 모델 학습 완료")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002928 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 88705, number of used features: 15
[LightGBM] [Info] Start training from score 2080467.000000
Fold 1 MAE: 126,532
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006031 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2753
[LightGBM] [Info] Number of data points in the train set: 177406, number of used features: 15
[LightGBM] [Info] Start training from score 2119460.000000
Fold 2 MAE: 318,519
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005529 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2790
[Li

# 2. 배달, 한식, 저장

In [8]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# =========================
# 경로/설정
# =========================
HAN_PATH = "data/수원시 한식 데이터백업.csv"
DELIVERY_PATH = "data/수원시 배달 데이터백업.csv"

OUT_DIR = "data/models"
os.makedirs(OUT_DIR, exist_ok=True)

DATE_COL = "TA_YMD"
TARGET = "AMT"

# =========================
# 유틸: 날씨 컬럼 자동 탐지 (temp/rain)
# =========================
def find_weather_cols(df, mode: str):
    cols = list(df.columns)
    low = {c: str(c).lower() for c in cols}

    if mode == "temp":
        keys = ["temp", "tavg", "tmean", "기온", "평균기온", "최고기온", "최저기온"]
    elif mode == "rain":
        keys = ["rain", "precip", "prcp", "강수", "강수량", "강우", "mm"]
    else:
        raise ValueError("mode must be 'temp' or 'rain'")

    picked = []
    for c in cols:
        s = low[c]
        if any(k in s for k in keys):
            picked.append(c)

    # 숫자형만 유지
    picked = [c for c in picked if pd.api.types.is_numeric_dtype(df[c])]
    return picked

# =========================
# 피처 생성
#  - 날짜 파생(연/월/일/요일 등)
#  - 동(DONG) 있으면 category로 사용
#  - lag/rolling (동별이면 groupby 적용)
# =========================
def make_features(df: pd.DataFrame, group_col: str | None = "DONG"):
    d = df.copy()

    if DATE_COL not in d.columns:
        raise ValueError(f"날짜 컬럼 {DATE_COL} 없음")
    if TARGET not in d.columns:
        raise ValueError(f"타겟 컬럼 {TARGET} 없음")

    d[DATE_COL] = pd.to_datetime(d[DATE_COL])
    d = d.sort_values(DATE_COL).reset_index(drop=True)

    # 날짜 파생 (datetime dtype 제거 목적)
    d["year"] = d[DATE_COL].dt.year
    d["month"] = d[DATE_COL].dt.month
    d["day"] = d[DATE_COL].dt.day
    d["dow"] = d[DATE_COL].dt.weekday
    d["weekend"] = (d["dow"] >= 5).astype(int)

    cat_cols = []
    if group_col and group_col in d.columns:
        d[group_col] = d[group_col].astype("category")
        cat_cols.append(group_col)
    else:
        group_col = None

    # Lag/Rolling
    def add_lag_roll(x):
        for lag in [1, 7, 14]:
            x[f"lag_{lag}"] = x[TARGET].shift(lag)
        for win in [7, 14]:
            x[f"roll_mean_{win}"] = x[TARGET].shift(1).rolling(win).mean()
            x[f"roll_std_{win}"] = x[TARGET].shift(1).rolling(win).std()
        return x

    if group_col:
        d = d.groupby(group_col, group_keys=False).apply(add_lag_roll)
    else:
        d = add_lag_roll(d)

    # object 남아있으면 category로 (LightGBM dtype 에러 방지)
    for c in d.columns:
        if d[c].dtype == "object" and c != TARGET:
            d[c] = d[c].astype("category")
            if c not in cat_cols:
                cat_cols.append(c)

    return d, cat_cols, group_col

# =========================
# 학습 + 최고 성능 모델 저장
#  - TS CV 4 folds
#  - 각 fold의 best_iteration_ 평균으로 최종 모델 재학습
#  - CV 평균 MAE가 더 낮으면 파일 덮어쓰기
# =========================
def train_and_save_best(df: pd.DataFrame, name: str, mode: str):
    """
    name: 저장 파일 prefix (예: 'han', 'delivery')
    mode: 'temp' or 'rain' (컬럼 자동 탐지용)
    """
    weather_cols = find_weather_cols(df, mode=mode)

    d, cat_cols, group_col = make_features(df, group_col="DONG")

    # 학습 피처: 타겟/날짜 제외 전부 (날씨 컬럼이 포함되어 있으면 자동 포함)
    drop_cols = [TARGET, DATE_COL]
    feature_cols = [c for c in d.columns if c not in drop_cols]

    # NA 제거
    X = d[feature_cols].copy()
    y = d[TARGET].copy()
    mask = ~X.isna().any(axis=1) & ~y.isna()
    X = X[mask].reset_index(drop=True)
    y = y[mask].reset_index(drop=True)

    # TS CV
    tscv = TimeSeriesSplit(n_splits=4)
    maes, best_iters = [], []

    params = dict(
        objective="regression_l1",
        n_estimators=3500,          # early stopping으로 자동 컷
        learning_rate=0.03,
        num_leaves=64,
        min_child_samples=30,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.3,
        reg_lambda=0.3,
        n_jobs=-1,
        random_state=42
    )

    print(f"\n===== [{name}] 학습 시작 (mode={mode}) =====")
    if len(weather_cols) == 0:
        print(f"⚠️ [{name}] {mode} 관련 날씨 컬럼 자동 탐지 실패(그래도 학습은 진행).")
    else:
        print(f"[{name}] 핵심 날씨 컬럼 후보: {weather_cols}")

    for fold, (tr, va) in enumerate(tscv.split(X), start=1):
        model = lgb.LGBMRegressor(**params)
        model.fit(
            X.iloc[tr], y.iloc[tr],
            eval_set=[(X.iloc[va], y.iloc[va])],
            eval_metric="l1",
            categorical_feature=cat_cols if len(cat_cols) else "auto",
            callbacks=[lgb.early_stopping(200, verbose=False)]
        )
        pred = model.predict(X.iloc[va])
        mae = mean_absolute_error(y.iloc[va], pred)
        maes.append(mae)
        best_iters.append(model.best_iteration_)
        print(f"[{name}] Fold {fold} MAE: {mae:,.0f}  (best_iter={model.best_iteration_})")

    cv_mae = float(np.mean(maes))
    final_estimators = int(np.clip(np.mean(best_iters), 300, 3500))
    print(f"[{name}] CV 평균 MAE: {cv_mae:,.0f} / 최종 n_estimators={final_estimators}")

    # 최종 모델 재학습
    final_model = lgb.LGBMRegressor(**{**params, "n_estimators": final_estimators})
    final_model.fit(X, y, categorical_feature=cat_cols if len(cat_cols) else "auto")

    # 저장 경로
    model_path = os.path.join(OUT_DIR, f"{name}_best_model.pkl")
    meta_path  = os.path.join(OUT_DIR, f"{name}_best_meta.pkl")

    # 기존 저장 모델이 있으면 성능 비교 후 더 좋을 때만 덮어쓰기
    should_save = True
    if os.path.exists(meta_path):
        try:
            old_meta = joblib.load(meta_path)
            old_mae = float(old_meta.get("cv_mae", np.inf))
            if cv_mae >= old_mae:
                should_save = False
                print(f"[{name}] 기존 모델이 더 좋거나 동일함: old_CV_MAE={old_mae:,.0f} <= new_CV_MAE={cv_mae:,.0f}")
        except Exception:
            # 메타가 깨져있으면 새로 저장
            should_save = True

    if should_save:
        joblib.dump(final_model, model_path)
        meta = {
            "cv_mae": cv_mae,
            "date_col": DATE_COL,
            "target_col": TARGET,
            "feature_cols": feature_cols,
            "cat_cols": cat_cols,
            "group_col": group_col,
            "mode": mode,
            "weather_cols_detected": weather_cols,
            "n_estimators": final_estimators,
        }
        joblib.dump(meta, meta_path)
        print(f"✅ [{name}] 최고 성능 모델 저장 완료")
        print(f" - model: {model_path}")
        print(f" - meta : {meta_path}")

    return final_model, cv_mae

# =========================
# 실행
# =========================
han_df = pd.read_csv(HAN_PATH)
del_df = pd.read_csv(DELIVERY_PATH)

# 한식 = 기온 중심
han_model, han_cv_mae = train_and_save_best(han_df, name="han", mode="temp")

# 배달 = 강수 중심
del_model, del_cv_mae = train_and_save_best(del_df, name="delivery", mode="rain")

print("\n===== 전체 완료 =====")
print(f"한식 CV MAE: {han_cv_mae:,.0f}")
print(f"배달 CV MAE: {del_cv_mae:,.0f}")



===== [han] 학습 시작 (mode=temp) =====
[han] 핵심 날씨 컬럼 후보: ['TEMP']
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2711
[LightGBM] [Info] Number of data points in the train set: 88583, number of used features: 15
[LightGBM] [Info] Start training from score 2083101.000000
[han] Fold 1 MAE: 121,904  (best_iter=3499)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2753
[LightGBM] [Info] Number of data points in the train set: 177164, number of used features: 15
[LightGBM] [Info] Start training from score 2120076.500000
[han] Fold 2 MAE: 297,366  (best_iter=3500)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010703 seconds.
You can set `force_col_wise=true` to remove th

✅ [delivery] 최고 성능 모델 저장 완료
 - model: data/models\delivery_best_model.pkl
 - meta : data/models\delivery_best_meta.pkl

===== 전체 완료 =====
한식 CV MAE: 204,039
배달 CV MAE: 6,889


In [11]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# =====================
# 경로
# =====================
HAN_MODEL_PATH = r"data/models/han_best_model.pkl"
HAN_META_PATH  = r"data/models/han_best_meta.pkl"
HAN_DATA_PATH  = r"data/수원시 한식 데이터백업.csv"   # 네 환경 경로로 맞춰

DEL_MODEL_PATH = r"data/models/delivery_best_model.pkl"
DEL_META_PATH  = r"data/models/delivery_best_meta.pkl"
DEL_DATA_PATH  = r"data/수원시 배달 데이터백업.csv"  # 네 환경 경로로 맞춰

# =====================
# 공통: 피처 생성 (학습 코드와 동일해야 함)
# =====================
DATE_COL = "TA_YMD"
TARGET = "AMT"

def make_features(df: pd.DataFrame, group_col: str | None = "DONG"):
    d = df.copy()
    d[DATE_COL] = pd.to_datetime(d[DATE_COL])
    d = d.sort_values(DATE_COL).reset_index(drop=True)

    d["year"] = d[DATE_COL].dt.year
    d["month"] = d[DATE_COL].dt.month
    d["day"] = d[DATE_COL].dt.day
    d["dow"] = d[DATE_COL].dt.weekday
    d["weekend"] = (d["dow"] >= 5).astype(int)

    cat_cols = []
    if group_col and group_col in d.columns:
        d[group_col] = d[group_col].astype("category")
        cat_cols.append(group_col)
    else:
        group_col = None

    def add_lag_roll(x):
        for lag in [1, 7, 14]:
            x[f"lag_{lag}"] = x[TARGET].shift(lag)
        for win in [7, 14]:
            x[f"roll_mean_{win}"] = x[TARGET].shift(1).rolling(win).mean()
            x[f"roll_std_{win}"] = x[TARGET].shift(1).rolling(win).std()
        return x

    if group_col:
        d = d.groupby(group_col, group_keys=False).apply(add_lag_roll)
    else:
        d = add_lag_roll(d)

    # object -> category (LightGBM dtype 방지)
    for c in d.columns:
        if d[c].dtype == "object" and c not in [TARGET]:
            d[c] = d[c].astype("category")
            if c not in cat_cols:
                cat_cols.append(c)

    return d

def wmape(y_true, y_pred, eps=1e-9):
    denom = np.sum(np.abs(y_true)) + eps
    return np.sum(np.abs(y_true - y_pred)) / denom

# =====================
# 평가 함수
# =====================
def evaluate_saved_model(model_path, meta_path, data_path, holdout_ratio=0.2, print_by_dong=True):
    model = joblib.load(model_path)
    meta = joblib.load(meta_path)

    feature_cols = meta["feature_cols"]
    cat_cols = meta.get("cat_cols", [])
    group_col = meta.get("group_col", None)

    df = pd.read_csv(data_path)
    d = make_features(df, group_col=group_col)

    # 학습 때처럼 NA 제거
    X = d[feature_cols].copy()
    y = d[TARGET].copy()
    mask = ~X.isna().any(axis=1) & ~y.isna()
    X = X[mask].reset_index(drop=True)
    y = y[mask].reset_index(drop=True)

    # 시계열 holdout: 뒤쪽 20%를 test로
    n = len(X)
    split = int(n * (1 - holdout_ratio))
    X_train, X_test = X.iloc[:split], X.iloc[split:]
    y_train, y_test = y.iloc[:split], y.iloc[split:]

    # 예측
    pred = model.predict(X_test)

    # 지표
    mae = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)
    r2 = r2_score(y_test, pred)
    w = wmape(y_test.values, pred)

    print(f"\n==== 평가: {model_path} ====")
    print(f"Samples: train={len(X_train):,} / test={len(X_test):,}")
    print(f"MAE  : {mae:,.0f}")
    print(f"RMSE : {rmse:,.0f}")
    print(f"R2   : {r2:.4f}")
    print(f"WMAPE: {w:.4%}")

        # (선택) 동별 MAE
    if print_by_dong and group_col and group_col in d.columns:
        d_masked = d.loc[mask].reset_index(drop=True)
        d_test = d_masked.iloc[split:].copy()
        d_test["pred"] = pred

        # 샘플 수가 1 이상인 동만
        grp = d_test.groupby(group_col, observed=True)
        stats = grp.apply(lambda g: pd.Series({
            "n": len(g),
            "mae": mean_absolute_error(g[TARGET].values, g["pred"].values) if len(g) > 0 else np.nan,
            "wmape": (np.sum(np.abs(g[TARGET].values - g["pred"].values)) / (np.sum(np.abs(g[TARGET].values)) + 1e-9)) if len(g) > 0 else np.nan
        }))
        stats = stats.dropna().sort_values("mae")

        if len(stats) == 0:
            print("\n[동별 MAE] 테스트 구간에 유효한 동 데이터가 없습니다. (표본 부족)")
        else:
            print("\n[동별 MAE TOP 10 (좋은 순)]")
            print(stats.head(10))
            print("\n[동별 MAE WORST 10 (나쁜 순)]")
            print(stats.tail(10))

# =====================
# 실행
# =====================
han_res = evaluate_saved_model(HAN_MODEL_PATH, HAN_META_PATH, HAN_DATA_PATH)
del_res = evaluate_saved_model(DEL_MODEL_PATH, DEL_META_PATH, DEL_DATA_PATH)



==== 평가: data/models/han_best_model.pkl ====
Samples: train=354,325 / test=88,582
MAE  : 151,157
RMSE : 1,679,193
R2   : 0.9814
WMAPE: 2.8072%

[동별 MAE TOP 10 (좋은 순)]
                   n           mae     wmape
DONG                                        
수원시 권선구 세류1동  1444.0   5575.550229  0.071540
수원시 권선구 세류3동  2065.0  13811.382568  0.010179
수원시 영통구 매탄4동  2043.0  13914.801772  0.010554
수원시 장안구 정자1동  2028.0  14649.504522  0.008968
수원시 장안구 조원2동  1470.0  15033.683022  0.014717
수원시 팔달구 화서1동  2055.0  17188.697282  0.009409
수원시 권선구 입북동   1865.0  21462.478350  0.015297
수원시 팔달구 매교동   2039.0  21764.044537  0.024374
수원시 팔달구 고등동   2056.0  22144.977556  0.010232
수원시 권선구 권선2동  2062.0  22380.597920  0.011487

[동별 MAE WORST 10 (나쁜 순)]
                   n           mae     wmape
DONG                                        
수원시 영통구 매탄2동  2028.0  9.722877e+04  0.063856
수원시 권선구 권선1동  2070.0  9.830294e+04  0.015604
수원시 영통구 매탄3동  2060.0  1.079290e+05  0.015677
수원시 영통구 광교1동  2059.0  1.771252e+05  0.016

# 3. 매출 데이터 바탕으로 모델 사용

## 점유율

In [29]:
# -*- coding: utf-8 -*-
import re
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# =====================================================
# 상수 / 경로
# =====================================================
DATE_COL = "TA_YMD"
TARGET = "AMT"

MY_DATE_COL = "ta_ymd"
MY_AMT_COL = "store_amt"

GRID_XLSX_PATH = "data/수원시 격자.xlsx"

HAN_DATA_PATH = "data/수원시 한식 데이터백업.csv"
DEL_DATA_PATH = "data/수원시 배달 데이터백업.csv"

HAN_MODEL_PATH = "data/models/han_best_model.pkl"
DEL_MODEL_PATH = "data/models/delivery_best_model.pkl"

HAN_META_PATH = "data/models/han_best_meta.pkl"
DEL_META_PATH = "data/models/delivery_best_meta.pkl"


# =====================================================
# 0) 수원시 격자 로드
# =====================================================
def load_suwon_grid_map(xlsx_path=GRID_XLSX_PATH):
    df = pd.read_excel(xlsx_path, header=None)
    df = df.dropna(subset=[0, 1, 2])

    grid = {}
    for _, r in df.iterrows():
        try:
            grid[str(r[0]).strip()] = (int(r[1]), int(r[2]))
        except:
            continue

    if not grid:
        raise ValueError("격자 엑셀에서 nx, ny를 읽지 못했습니다.")
    return grid


def resolve_nxny_from_input_dong(dong, grid_map):
    if dong not in grid_map:
        raise ValueError(f"'{dong}'이(가) 격자 엑셀에 없습니다.")
    return grid_map[dong][0], grid_map[dong][1]


# =====================================================
# 1) 기상청 단기예보 + fallback (과거 평균 날씨)
# =====================================================
def _parse_pcp(v):
    if v in [None, "강수없음", "-", "없음"]:
        return 0.0
    s = str(v).replace("mm", "").strip()
    try:
        return float(s)
    except:
        nums = re.findall(r"[\d.]+", s)
        return float(nums[0]) if nums else 0.0


def fetch_daily_weather_kma(
    nx, ny, target_date, RAIN_ID, base_df, dong
):
    """
    1) 기상청 단기예보 가능 → 사용
    2) NO_DATA / 과거·미래 → 과거 평균 날씨 대체
    """

    d = datetime.strptime(target_date, "%Y-%m-%d").date()
    base_date = (d - timedelta(days=1)).strftime("%Y%m%d")

    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": RAIN_ID,
        "dataType": "JSON",
        "numOfRows": 3000,
        "pageNo": 1,
        "base_date": base_date,
        "base_time": "2300",
        "nx": nx,
        "ny": ny,
    }

    try:
        r = requests.get(url, params=params, timeout=10)
        js = r.json()
        body = js.get("response", {}).get("body")

        if body:
            items = body.get("items", {}).get("item", [])
            want = d.strftime("%Y%m%d")

            temps, rains = [], []
            for it in items:
                if it.get("fcstDate") != want:
                    continue
                if it.get("category") == "TMP":
                    temps.append(float(it.get("fcstValue")))
                elif it.get("category") == "PCP":
                    rains.append(_parse_pcp(it.get("fcstValue")))

            if temps:
                return {
                    "temp_mean": float(np.mean(temps)),
                    "rain_mean": float(np.mean(rains)) if rains else 0.0,
                    "rain_peak": float(np.max(rains)) if rains else 0.0,
                    "source": "기상청 단기예보",
                }
    except:
        pass

    # =============================
    # fallback: 과거 평균 날씨
    # =============================
    hist = base_df.copy()
    hist[DATE_COL] = pd.to_datetime(hist[DATE_COL])
    hist = hist[hist["DONG"] == dong]

    temp_cols = [c for c in hist.columns if "temp" in c.lower()]
    rain_cols = [c for c in hist.columns if "rain" in c.lower()]

    return {
        "temp_mean": float(hist[temp_cols[0]].mean()) if temp_cols else np.nan,
        "rain_mean": float(hist[rain_cols[0]].mean()) if rain_cols else 0.0,
        "rain_peak": float(hist[rain_cols[0]].max()) if rain_cols else 0.0,
        "source": "과거 평균 날씨(대체)",
    }


# =====================================================
# 2) 공휴일 (천문연구원)
# =====================================================
def fetch_holiday_flag_kasi(target_date, HOLIDAY_ID):
    d = datetime.strptime(target_date, "%Y-%m-%d")
    url = "https://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo"
    params = {
        "serviceKey": HOLIDAY_ID,
        "solYear": d.year,
        "solMonth": f"{d.month:02d}",
        "_type": "json",
    }

    try:
        r = requests.get(url, params=params, timeout=10)
        items = (
            r.json()
            .get("response", {})
            .get("body", {})
            .get("items", {})
            .get("item", [])
        )

        if isinstance(items, dict):
            items = [items]

        for it in items:
            if str(it.get("locdate")) == d.strftime("%Y%m%d"):
                return {"is_holiday": 1}
    except:
        pass

    return {"is_holiday": 0}


# =====================================================
# 3) 예측용 feature 생성 (❗ DONG 제거됨)
# =====================================================
def build_feature_row_for_prediction(
    base_df, feature_cols, dong, target_date, weather, is_holiday
):
    d = base_df.copy()
    d[DATE_COL] = pd.to_datetime(d[DATE_COL])
    d = d[d["DONG"] == dong].sort_values(DATE_COL)

    td = pd.to_datetime(target_date)
    hist = d[d[DATE_COL] < td]

    row = {
        "year": td.year,
        "month": td.month,
        "day": td.day,
        "dow": td.weekday(),
        "weekend": int(td.weekday() >= 5),
    }

    for lag in [1, 7, 14]:
        row[f"lag_{lag}"] = hist[TARGET].iloc[-lag] if len(hist) >= lag else np.nan

    for win in [7, 14]:
        row[f"roll_mean_{win}"] = hist[TARGET].tail(win).mean() if len(hist) >= win else np.nan
        row[f"roll_std_{win}"] = hist[TARGET].tail(win).std() if len(hist) >= win else np.nan

    row["temp_mean"] = weather["temp_mean"]
    row["rain_mean"] = weather["rain_mean"]
    row["rain_peak"] = weather["rain_peak"]
    row["is_holiday"] = is_holiday

    return pd.DataFrame([{c: row.get(c, np.nan) for c in feature_cols}])


# =====================================================
# 4) 점유율 + 보정계수
# =====================================================
def compute_store_share_and_adj(
    my_store_df, base_df, dong, target_date, lookback_days=60
):
    d0 = datetime.strptime(target_date, "%Y-%m-%d").date()
    start = d0 - timedelta(days=lookback_days)

    ms = my_store_df.copy()
    ms[MY_DATE_COL] = pd.to_datetime(ms[MY_DATE_COL])
    store_amt_60 = ms[
        (ms[MY_DATE_COL].dt.date >= start) &
        (ms[MY_DATE_COL].dt.date < d0)
    ][MY_AMT_COL].sum()

    bd = base_df.copy()
    bd[DATE_COL] = pd.to_datetime(bd[DATE_COL])
    dong_amt_60 = bd[
        (bd["DONG"] == dong) &
        (bd[DATE_COL].dt.date >= start) &
        (bd[DATE_COL].dt.date < d0)
    ][TARGET].sum()

    share = (store_amt_60 / dong_amt_60 * 100) if dong_amt_60 > 0 else 0.0
    return share, 1.0


# =====================================================
# 5) 거래건수 추정
# =====================================================
def estimate_txn_count(my_store_df, target_date, pred_store_amt):
    pseudo_ticket = max(15000.0, pred_store_amt / 20.0)
    return int(pred_store_amt / pseudo_ticket)


# =====================================================
# 6) 메인 함수
# =====================================================
def predict_my_store_sales(
    my_store_csv,
    dong,
    date_str,
    service_type,
    RAIN_ID,
    HOLIDAY_ID,
    grid_xlsx_path=GRID_XLSX_PATH,
):
    if service_type == "delivery":
        model_path, meta_path, data_path = (
            DEL_MODEL_PATH, DEL_META_PATH, DEL_DATA_PATH
        )
    else:
        model_path, meta_path, data_path = (
            HAN_MODEL_PATH, HAN_META_PATH, HAN_DATA_PATH
        )

    model = joblib.load(model_path)
    meta = joblib.load(meta_path)

    # ❗ DONG 제거
    feature_cols = [c for c in meta["feature_cols"] if c != "DONG"]

    base_df = pd.read_csv(data_path)
    my_store_df = pd.read_csv(my_store_csv)

    grid_map = load_suwon_grid_map(grid_xlsx_path)
    nx, ny = resolve_nxny_from_input_dong(dong, grid_map)

    weather = fetch_daily_weather_kma(
        nx, ny, date_str, RAIN_ID, base_df, dong
    )
    holi = fetch_holiday_flag_kasi(date_str, HOLIDAY_ID)

    X_pred = build_feature_row_for_prediction(
        base_df, feature_cols, dong, date_str, weather, holi["is_holiday"]
    )

    pred_dong_amt = float(model.predict(X_pred)[0])
    share, adj = compute_store_share_and_adj(
        my_store_df, base_df, dong, date_str
    )

    pred_store_amt = pred_dong_amt * (share / 100.0) * adj
    pred_cnt = estimate_txn_count(my_store_df, date_str, pred_store_amt)

    # =========================
    # 출력
    # =========================
    print(f"📍지역(동): {dong}")
    print(f"📅날짜: {date_str}")
    print(f"🌦️날씨(출처): {weather['source']}")
    print(f"   - 평균기온: {weather['temp_mean']:.1f}°C")
    print(f"   - 강수(평균/피크): {weather['rain_mean']:.2f} / {weather['rain_peak']:.2f} mm")
    print(f"👥예상 매출건수: {pred_cnt}건")
    print(f"💰예상 동 전체 일매출: {pred_dong_amt:,.0f} 원")
    print(f"🏪가게 점유율(최근 60일): {share:.2f}%")
    print(f"🏪예상 가게 일매출: {pred_store_amt:,.0f} 원")


## 사용예시

In [34]:
import os
from dotenv import load_dotenv
load_dotenv()

holiday_key_env = os.getenv("HOLIDAY_ID")
rain_key_env = os.getenv("RAIN_ID")

predict_my_store_sales(
    my_store_csv="data/my_store.csv",
    dong="행궁동",
    date_str="2025-12-30",
    service_type="han",
    RAIN_ID=rain_key_env,
    HOLIDAY_ID=holiday_key_env,
    grid_xlsx_path="data/수원시 격자.xlsx"
)


📍지역(동): 행궁동
📅날짜: 2025-12-30
🌦️날씨(출처): 과거 평균 날씨(대체)
   - 평균기온: 13.4°C
   - 강수(평균/피크): 0.16 / 37.30 mm
👥예상 매출건수: 0건
💰예상 동 전체 일매출: 84,208 원
🏪가게 점유율(최근 60일): 0.00%
🏪예상 가게 일매출: 0 원


# 4. 모델 재학습

In [27]:
# -*- coding: utf-8 -*-
import os
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import timedelta

# =====================================================
# 설정
# =====================================================
DATE_COL = "TA_YMD"
TARGET = "AMT"

HAN_DATA_PATH = "data/수원시 한식 데이터백업.csv"
DEL_DATA_PATH = "data/수원시 배달 데이터백업.csv"

OUT_DIR = "data/models"
os.makedirs(OUT_DIR, exist_ok=True)

# =====================================================
# 공통 전처리
# =====================================================
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])
    df = df.sort_values(DATE_COL).reset_index(drop=True)

    # ---------- 날짜 파생 ----------
    df["year"] = df[DATE_COL].dt.year
    df["month"] = df[DATE_COL].dt.month
    df["day"] = df[DATE_COL].dt.day
    df["dow"] = df[DATE_COL].dt.weekday
    df["weekend"] = (df["dow"] >= 5).astype(int)

    # ---------- DONG 제거 ----------
    if "DONG" in df.columns:
        df = df.drop(columns=["DONG"])

    # ---------- lag / rolling ----------
    for lag in [1, 7, 14]:
        df[f"lag_{lag}"] = df[TARGET].shift(lag)

    for win in [7, 14]:
        df[f"roll_mean_{win}"] = df[TARGET].rolling(win).mean()
        df[f"roll_std_{win}"] = df[TARGET].rolling(win).std()

    df = df.dropna().reset_index(drop=True)
    return df


# =====================================================
# 학습 함수
# =====================================================
def train_and_save(
    data_path: str,
    model_name: str,
):
    print(f"\n===== TRAIN START: {model_name} =====")

    df = pd.read_csv(data_path)
    df = preprocess(df)

    feature_cols = [c for c in df.columns if c not in [DATE_COL, TARGET]]

    # ---------- time split ----------
    split_date = df[DATE_COL].quantile(0.8)
    train_df = df[df[DATE_COL] <= split_date]
    valid_df = df[df[DATE_COL] > split_date]

    X_train = train_df[feature_cols]
    y_train = train_df[TARGET]
    X_val = valid_df[feature_cols]
    y_val = valid_df[TARGET]

    model = lgb.LGBMRegressor(
        objective="regression_l1",
        n_estimators=5000,
        learning_rate=0.02,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_samples=30,
        random_state=42,
        n_jobs=-1,
    )

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="l1",
        callbacks=[lgb.early_stopping(300, verbose=True)],
    )

    pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, pred)

    print(f"✅ VALID MAE = {mae:,.2f}")

    # ---------- 저장 ----------
    model_path = f"{OUT_DIR}/{model_name}_best_model.pkl"
    meta_path = f"{OUT_DIR}/{model_name}_best_meta.pkl"

    joblib.dump(model, model_path)
    joblib.dump(
        {
            "feature_cols": feature_cols,
            "valid_mae": mae,
            "n_train": len(train_df),
            "n_valid": len(valid_df),
        },
        meta_path,
    )

    print(f"💾 model saved: {model_path}")
    print(f"💾 meta  saved: {meta_path}")

    return mae


# =====================================================
# 실행
# =====================================================
if __name__ == "__main__":
    han_mae = train_and_save(HAN_DATA_PATH, "han")
    del_mae = train_and_save(DEL_DATA_PATH, "delivery")

    print("\n===== DONE =====")
    print(f"HAN MAE     : {han_mae:,.2f}")
    print(f"DELIVERY MAE: {del_mae:,.2f}")



===== TRAIN START: han =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009802 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2760
[LightGBM] [Info] Number of data points in the train set: 355094, number of used features: 14
[LightGBM] [Info] Start training from score 2108030.000000
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l1: 210080
✅ VALID MAE = 210,080.03
💾 model saved: data/models/han_best_model.pkl
💾 meta  saved: data/models/han_best_meta.pkl

===== TRAIN START: delivery =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001237 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2500
[LightGBM] [Info] Number of data points in the train set: 30234, number of used features: 14
[LightGBM] [Info] Start training from score

# 5. 정님 모델 수정

In [2]:
import os
import json
import time
import warnings
import joblib
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.base import clone
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

# =============================
# XGB 사용 가능 여부 체크
# =============================
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except ImportError:
    HAS_XGB = False


# =============================
# Config
# =============================
HANSIK_PATH = "data/수원시 한식 데이터백업.csv"
DELIV_PATH  = "data/수원시 배달 데이터백업.csv"

SAVE_DIR = "data/model"           
MAX_HOURS = list(range(1, 11))    # 1~10
TEST_RATIO = 0.2                  # 마지막 20% 테스트
RANDOM_STATE = 42

# 원핫 폭발 방지(가능한 sklearn 버전에서만 적용)
OHE_MIN_FREQUENCY = 20
OHE_MAX_CATEGORIES = 200

# UNIT lag/rolling
LAGS = [1, 7, 14]
ROLL_WINDOWS = [7, 14]

# XGB 파라미터(너무 느리면 max_depth/early stopping 줄이세요)
XGB_PARAMS = dict(
    n_estimators=5000,          # 크게 주고 early stopping으로 컷
    learning_rate=0.03,
    max_depth=8,
    min_child_weight=2,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="reg:absoluteerror",  # MAE 직접 최적화(지원 버전 필요)
    tree_method="hist",
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

ET_PARAMS = dict(
    n_estimators=800,
    max_depth=None,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

# 누수 위험(예측 시점에 모르는 값) 기본 제외
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]


# =============================
# Utils
# =============================
def evaluate_reg(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return float(mae), float(rmse), float(r2)


def safe_onehot_encoder():
    """
    sklearn 버전에 따라 min_frequency / max_categories 미지원일 수 있어 예외 처리.
    """
    try:
        return OneHotEncoder(
            handle_unknown="ignore",
            min_frequency=OHE_MIN_FREQUENCY,
            max_categories=OHE_MAX_CATEGORIES
        )
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore")


def build_preprocess(X: pd.DataFrame):
    """
    X의 dtype 기반으로 ColumnTransformer 생성
    """
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", safe_onehot_encoder())
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols)
        ],
        remainder="drop"
    )
    return preprocess


# =============================
# 1) Load
# =============================
hansik = pd.read_csv(HANSIK_PATH)
deliv  = pd.read_csv(DELIV_PATH)

required = {"TA_YMD", "DONG", "HOUR", "DAY", "UNIT"}
missing = required - set(hansik.columns)
if missing:
    raise ValueError(f"[한식 데이터] 필수 컬럼이 없습니다: {missing}")

missing2 = {"TA_YMD", "DONG", "HOUR", "DAY"} - set(deliv.columns)
if missing2:
    raise ValueError(f"[배달 데이터] 필수 컬럼이 없습니다: {missing2}")


# =============================
# 2) Merge DELIV as features
# =============================
merge_keys = ["TA_YMD", "DONG", "HOUR", "DAY"]

deliv_feat = deliv.copy()
for c in deliv_feat.columns:
    if c not in merge_keys:
        deliv_feat.rename(columns={c: f"DELIV_{c}"}, inplace=True)

df = hansik.merge(deliv_feat, on=merge_keys, how="left")


# =============================
# 3) Date features
# =============================
df["TA_YMD"] = df["TA_YMD"].astype(str)
df["DATE"] = pd.to_datetime(df["TA_YMD"], format="%Y%m%d", errors="coerce")
df = df[df["DATE"].notna()].copy()

df["YEAR"] = df["DATE"].dt.year
df["MONTH"] = df["DATE"].dt.month
df["DAY_OF_MONTH"] = df["DATE"].dt.day
df["WEEKOFYEAR"] = df["DATE"].dt.isocalendar().week.astype(int)


# =============================
# 4) Lag/Rolling features for UNIT (no leakage if time-split)
#    group key: (DONG, HOUR)
# =============================
df = df.sort_values(["DONG", "HOUR", "DATE"]).reset_index(drop=True)
grp = df.groupby(["DONG", "HOUR"], sort=False)

for lag in LAGS:
    df[f"UNIT_LAG_{lag}"] = grp["UNIT"].shift(lag)

for w in ROLL_WINDOWS:
    df[f"UNIT_ROLL_MEAN_{w}"] = grp["UNIT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).mean()
    df[f"UNIT_ROLL_STD_{w}"]  = grp["UNIT"].shift(1).rolling(window=w, min_periods=max(2, w // 3)).std()

df["UNIT_LAG_1_MISSING"] = df["UNIT_LAG_1"].isna().astype(int)

# 타겟
y_all = df["UNIT"].copy()

# 피처
X_all = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")


# =============================
# 5) Train per HOUR
# =============================
os.makedirs(SAVE_DIR, exist_ok=True)

hours = sorted(pd.unique(df["HOUR"].dropna()))
hours = [int(h) for h in hours if int(h) in MAX_HOURS]

print(f"[INFO] XGB 사용 여부: {HAS_XGB}")
print(f"[INFO] 학습 시간대: {hours}")
print(f"[INFO] 저장 폴더: {SAVE_DIR}")

all_results = []
t0 = time.time()

for h in hours:
    idx = (df["HOUR"].astype(int) == h)
    X_h = X_all.loc[idx].copy()
    y_h = y_all.loc[idx].copy()
    d_h = df.loc[idx, "DATE"].copy()

    if len(X_h) < 800:
        print(f"\n[SKIP] HOUR={h:02d}: 데이터가 너무 적음 (n={len(X_h)})")
        continue

    # 날짜 정렬
    order = np.argsort(d_h.values)
    X_h = X_h.iloc[order].reset_index(drop=True)
    y_h = y_h.iloc[order].reset_index(drop=True)

    n = len(X_h)
    split = int(n * (1 - TEST_RATIO))
    X_train, X_test = X_h.iloc[:split], X_h.iloc[split:]
    y_train, y_test = y_h.iloc[:split], y_h.iloc[split:]

    # XGB early stopping용 valid (train의 마지막 10%)
    split2 = int(len(X_train) * 0.9)
    X_tr, X_val = X_train.iloc[:split2], X_train.iloc[split2:]
    y_tr, y_val = y_train.iloc[:split2], y_train.iloc[split2:]

    print(f"\n===== HOUR={h:02d} | train={len(X_train)} test={len(X_test)} =====")

    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{h:02d}")
    os.makedirs(hour_dir, exist_ok=True)

    candidates = []

    # -------------------------
    # A방식: preprocess를 먼저 fit/transform 후 모델 학습
    # -------------------------
    preprocess = build_preprocess(X_train)
    pre = clone(preprocess)

    X_tr_enc  = pre.fit_transform(X_tr)
    X_val_enc = pre.transform(X_val)
    X_test_enc = pre.transform(X_test)

    # --- 1) XGB (있으면 가장 강력) ---
    if HAS_XGB:
        # objective가 버전에 따라 에러날 수 있어 fallback 처리
        xgb_params = dict(XGB_PARAMS)
        try:
            model_xgb = XGBRegressor(**xgb_params)
            model_xgb.fit(
                X_tr_enc, y_tr,
                eval_set=[(X_val_enc, y_val)],
                verbose=False,
                early_stopping_rounds=200
            )
        except Exception as e:
            # fallback: objective를 reg:squarederror로
            xgb_params["objective"] = "reg:squarederror"
            model_xgb = XGBRegressor(**xgb_params)
            model_xgb.fit(
                X_tr_enc, y_tr,
                eval_set=[(X_val_enc, y_val)],
                verbose=False,
                early_stopping_rounds=200
            )

        pred = model_xgb.predict(X_test_enc)
        mae, rmse, r2 = evaluate_reg(y_test, pred)
        print(f"- XGB       | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")

        bundle = {"preprocess": pre, "model": model_xgb, "model_name": "XGB"}
        candidates.append(("XGB", bundle, mae, rmse, r2))

    # --- 2) ExtraTrees (빠른/안정 대안) ---
    model_et = ExtraTreesRegressor(**ET_PARAMS)
    model_et.fit(X_tr_enc, y_tr)  # valid는 사용 안 함(빠름)

    pred = model_et.predict(X_test_enc)
    mae, rmse, r2 = evaluate_reg(y_test, pred)
    print(f"- EXTRATREE | MAE={mae:,.3f} RMSE={rmse:,.3f} R2={r2:,.4f}")

    bundle = {"preprocess": pre, "model": model_et, "model_name": "EXTRATREE"}
    candidates.append(("EXTRATREE", bundle, mae, rmse, r2))

    # --- pick best by MAE ---
    candidates.sort(key=lambda x: x[2])
    best_name, best_bundle, best_mae, best_rmse, best_r2 = candidates[0]

    best_path = os.path.join(hour_dir, "best_model.joblib")
    joblib.dump(best_bundle, best_path)

    meta = {
        "HOUR": int(h),
        "BEST_MODEL": best_name,
        "MAE": float(best_mae),
        "RMSE": float(best_rmse),
        "R2": float(best_r2),
        "TRAIN_SIZE": int(len(X_train)),
        "TEST_SIZE": int(len(X_test)),
        "FEATURE_COLS": list(X_all.columns),
        "DROP_COLS": DROP_COLS,
        "LAGS": LAGS,
        "ROLL_WINDOWS": ROLL_WINDOWS,
        "HAS_XGB": bool(HAS_XGB),
        "XGB_PARAMS": XGB_PARAMS if HAS_XGB else None,
        "ET_PARAMS": ET_PARAMS,
    }
    with open(os.path.join(hour_dir, "meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"✅ BEST => {best_name} | MAE={best_mae:,.3f} 저장: {best_path}")

    all_results.append({
        "HOUR": int(h),
        "BEST_MODEL": best_name,
        "MAE": float(best_mae),
        "RMSE": float(best_rmse),
        "R2": float(best_r2),
        "SAVE_PATH": best_path
    })

# 결과 저장
res_df = pd.DataFrame(all_results).sort_values(["HOUR"])
res_path = os.path.join(SAVE_DIR, "results_by_hour.csv")
res_df.to_csv(res_path, index=False, encoding="utf-8-sig")

print("\n[DONE] 학습 완료")
print(f"- 결과표 저장: {res_path}")
print(f"- 모델 저장 폴더: {SAVE_DIR}")
print(f"- 총 소요(초): {time.time() - t0:,.1f}")


# =============================
# 6) 로드/예측 헬퍼
# =============================
def load_best_bundle(hour: int):
    hour_dir = os.path.join(SAVE_DIR, f"HOUR_{hour:02d}")
    path = os.path.join(hour_dir, "best_model.joblib")
    return joblib.load(path)

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)

"""
사용 예:
bundle = load_best_bundle(5)
sample = X_all.iloc[[0]].copy()          # 컬럼 구조 동일해야 함
yhat = predict_with_bundle(bundle, sample)
print(yhat)
"""


[INFO] XGB 사용 여부: True
[INFO] 학습 시간대: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[INFO] 저장 폴더: data/model

===== HOUR=01 | train=33935 test=8484 =====
- XGB       | MAE=17,040.274 RMSE=36,555.344 R2=0.1986
- EXTRATREE | MAE=18,416.771 RMSE=36,421.394 R2=0.2044
✅ BEST => XGB | MAE=17,040.274 저장: data/model\HOUR_01\best_model.joblib

===== HOUR=02 | train=33998 test=8500 =====
- XGB       | MAE=7,673.858 RMSE=28,665.482 R2=0.0246
- EXTRATREE | MAE=9,535.827 RMSE=28,756.635 R2=0.0184
✅ BEST => XGB | MAE=7,673.858 저장: data/model\HOUR_02\best_model.joblib

===== HOUR=03 | train=35715 test=8929 =====
- XGB       | MAE=8,009.915 RMSE=30,807.032 R2=-0.0119
- EXTRATREE | MAE=10,043.419 RMSE=30,236.351 R2=0.0253
✅ BEST => XGB | MAE=8,009.915 저장: data/model\HOUR_03\best_model.joblib

===== HOUR=04 | train=36360 test=9091 =====
- XGB       | MAE=4,418.536 RMSE=9,327.978 R2=0.2978
- EXTRATREE | MAE=4,300.277 RMSE=8,873.857 R2=0.3645
✅ BEST => EXTRATREE | MAE=4,300.277 저장: data/model\HOUR_04\best_model.joblib


'\n사용 예:\nbundle = load_best_bundle(5)\nsample = X_all.iloc[[0]].copy()          # 컬럼 구조 동일해야 함\nyhat = predict_with_bundle(bundle, sample)\nprint(yhat)\n'

# 6. 정님 모델

In [39]:
# -*- coding: utf-8 -*-
import os
import joblib
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()
SERVICE_KEY = os.getenv("RAIN_ID")

# =========================
# A방식 학습 저장 위치
# =========================
MODELS_ROOT = "data/model"
MODEL_BUNDLE_TEMPLATE = os.path.join(MODELS_ROOT, "HOUR_{hour:02d}", "best_model.joblib")

USE_LOG_TARGET = False
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]

# =========================
# 항목요약 시간대(01~10)
# =========================
HOUR_BINS = {
    1: list(range(0, 7)),
    2: [7, 8],
    3: [9, 10],
    4: [11, 12],
    5: [13, 14],
    6: [15, 16],
    7: [17, 18],
    8: [19, 20],
    9: [21, 22],
    10: [23],
}

# =========================
# 날짜 파생 + DAY(01=월..07=일)
# =========================
def compute_day_code(date_yyyymmdd: str) -> int:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return int(dt.weekday() + 1)

def make_date_features(date_yyyymmdd: str) -> dict:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return {
        "TA_YMD": str(date_yyyymmdd),
        "YEAR": int(dt.year),
        "MONTH": int(dt.month),
        "DAY_OF_MONTH": int(dt.day),
        "WEEKOFYEAR": int(dt.isocalendar().week),
    }

# =========================
# 기상청 단기예보 TMP/PCP
# =========================
def _parse_pcp_to_mm(x):
    if x is None:
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if s in ("강수없음", "없음", "", "nan", "NaN"):
        return 0.0
    if "미만" in s:
        try:
            num = float(s.replace("mm", "").replace("미만", "").strip())
            return max(0.0, num * 0.5)
        except:
            return 0.0
    if "~" in s:
        try:
            a, b = s.replace("mm", "").split("~")
            return (float(a) + float(b)) / 2.0
        except:
            return 0.0
    try:
        return float(s.replace("mm", ""))
    except:
        return 0.0

def fetch_vilage_fcst_json(service_key, base_date, base_time, nx, ny, num_rows=3000):
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_hourly_tmp_pcp_for_date(service_key, target_yyyymmdd, nx, ny):
    base_times = ["2300","2000","1700","1400","1100","0800","0500","0200"]
    today = datetime.now().strftime("%Y%m%d")
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")

    for bd in [today, yesterday]:
        for bt in base_times:
            try:
                js = fetch_vilage_fcst_json(service_key, bd, bt, nx, ny)
                items = js["response"]["body"]["items"]["item"]
                df = pd.DataFrame(items)

                df = df[df["category"].isin(["TMP", "PCP"])]
                df = df[df["fcstDate"].astype(str) == str(target_yyyymmdd)]
                if df.empty:
                    continue

                df["fcstTime"] = df["fcstTime"].astype(str).str.zfill(4)
                piv = df.pivot_table(index="fcstTime", columns="category", values="fcstValue", aggfunc="first").reset_index()

                if "TMP" not in piv.columns:
                    continue

                piv["TMP"] = piv["TMP"].astype(float)
                piv["PCP"] = piv["PCP"].apply(_parse_pcp_to_mm) if "PCP" in piv.columns else 0.0
                piv["PCP"] = piv["PCP"].astype(float)
                piv["base_used"] = f"{bd}-{bt}"
                return piv.sort_values("fcstTime").reset_index(drop=True)
            except:
                continue

    raise RuntimeError("해당 날짜의 기상청 예보 데이터를 찾지 못했습니다.")

def aggregate_weather_to_item_slots(hourly_df: pd.DataFrame) -> dict:
    hourly_df["HOUR_OF_DAY"] = hourly_df["fcstTime"].str[:2].astype(int)
    out = {}
    for h, hours in HOUR_BINS.items():
        sub = hourly_df[hourly_df["HOUR_OF_DAY"].isin(hours)]
        out[h] = {
            "TEMP": float(sub["TMP"].mean()) if not sub.empty else np.nan,
            "RAIN": float(sub["PCP"].sum()) if not sub.empty else 0.0,
        }

    temps = [v["TEMP"] for v in out.values() if not pd.isna(v["TEMP"])]
    fill_temp = float(np.mean(temps)) if temps else 0.0
    for h in out:
        if pd.isna(out[h]["TEMP"]):
            out[h]["TEMP"] = fill_temp
    return out

# =========================
# ✅ 핵심: 번들 로드 + 컬럼 align
# =========================
def load_best_bundle(hour: int) -> dict:
    path = MODEL_BUNDLE_TEMPLATE.format(hour=hour)
    if not os.path.exists(path):
        raise FileNotFoundError(f"best_model.joblib이 없습니다: {path}")
    bundle = joblib.load(path)
    if not isinstance(bundle, dict) or "preprocess" not in bundle or "model" not in bundle:
        raise ValueError(f"번들 포맷이 아닙니다(학습 A방식 저장물인지 확인): {path}")
    return bundle

def get_required_input_columns(preprocess) -> list:
    """
    ColumnTransformer가 fit될 때의 입력 컬럼명.
    sklearn >= 1.0 이면 feature_names_in_ 존재.
    없으면 transformers_에서 cols를 모아서 만든다.
    """
    if hasattr(preprocess, "feature_names_in_"):
        return [str(c) for c in preprocess.feature_names_in_]

    # fallback
    cols = []
    for _, _, c in getattr(preprocess, "transformers_", []):
        if c is None:
            continue
        if isinstance(c, (list, tuple, np.ndarray, pd.Index)):
            cols.extend(list(c))
    return sorted(set([str(x) for x in cols]))

def align_X_to_required_columns(X: pd.DataFrame, required_cols: list) -> pd.DataFrame:
    X2 = X.copy()

    # 없는 컬럼은 NaN으로 생성
    for c in required_cols:
        if c not in X2.columns:
            X2[c] = np.nan

    # 필요 없는 컬럼은 제거(안 해도 되지만 안전)
    X2 = X2[required_cols]
    return X2

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]

    required_cols = get_required_input_columns(pre)
    X_aligned = align_X_to_required_columns(X_new, required_cols)

    X_enc = pre.transform(X_aligned)
    return model.predict(X_enc)

# =========================
# 날짜 1개 → 1~10시간대 예측
# =========================
def predict_date_all_hours_with_weather(date_yyyymmdd: str, nx: int, ny: int, dong: str = None, service_key: str = SERVICE_KEY) -> pd.DataFrame:
    if not service_key:
        raise ValueError("RAIN_ID(기상청 서비스키)가 .env 또는 환경변수에 없습니다.")

    day_code = compute_day_code(date_yyyymmdd)
    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    rows = []

    for hour in range(1, 11):
        try:
            bundle = load_best_bundle(hour)
        except FileNotFoundError:
            continue

        row = {}
        row.update(make_date_features(date_yyyymmdd))
        row.update({
            "HOUR": hour,
            "DAY": day_code,
            "TEMP": weather_by_hour[hour]["TEMP"],
            "RAIN": weather_by_hour[hour]["RAIN"],
            "DELIV_TEMP": weather_by_hour[hour]["TEMP"],
            "DELIV_RAIN": weather_by_hour[hour]["RAIN"],
        })

        # ✅ 학습 때 DONG이 있었다면, 예측에도 넣어주는 게 성능/안정에 중요
        if dong is not None:
            row["DONG"] = str(dong)

        X_row = pd.DataFrame([row])
        X_row = X_row.drop(columns=[c for c in DROP_COLS if c in X_row.columns], errors="ignore")

        pred = float(predict_with_bundle(bundle, X_row)[0])
        if USE_LOG_TARGET:
            pred = float(np.expm1(pred))

        rows.append({
            "TA_YMD": str(date_yyyymmdd),
            "DAY": day_code,
            "HOUR": hour,
            "BEST_MODEL": bundle.get("model_name", "UNKNOWN"),
            "TEMP": float(row["TEMP"]),
            "RAIN": float(row["RAIN"]),
            "PRED_UNIT": int(round(pred)),
            "KMA_BASE_USED": str(hourly_weather.loc[0, "base_used"]),
        })

    if not rows:
        raise FileNotFoundError(f"{MODELS_ROOT}/HOUR_XX/best_model.joblib 파일을 찾지 못했습니다. 저장 경로를 확인하세요.")

    return pd.DataFrame(rows).sort_values("HOUR").reset_index(drop=True)

# =========================
# 사용 예시
# =========================
if __name__ == "__main__":
    df_pred = predict_date_all_hours_with_weather(
        date_yyyymmdd="20251229",
        nx=61,
        ny=121,
        dong="행궁동"   # ✅ 가능하면 꼭 넣으세요 (학습에 DONG이 있었기 때문)
    )
    print(df_pred)


     TA_YMD  DAY  HOUR BEST_MODEL  TEMP  RAIN  PRED_UNIT  KMA_BASE_USED
0  20251229    1     1        XGB  0.25   0.0      56506  20251229-2000
1  20251229    1     2        XGB  0.25   0.0      11251  20251229-2000
